In [1]:
import json
import pandas as pd

In [2]:
data_in = "../../sih-raw-data/geojson/"
data_out = "../data/geojson/"

In [3]:
with open(data_in + r"gadm36_IND_1.json", "r") as f:
    states = json.load(f)

In [4]:
for i in states["features"]:
    v = i["properties"]["NAME_1"].lower()
    if v == "nct of delhi":
        i["id"] = "delhi"
    elif v == "jammu and kashmir":
        i["id"] = "jammu & kashmir"
    elif v == "puducherry":
        i["id"] = "pondicherry"
    elif v == "andaman and nicobar":
        i["id"] = "andaman & nicobar"
    elif v == "dadar and nagar haveli":
        i["id"] = "dadar and nagar hav"
    else:
        i["id"] = v

In [5]:
with open(data_out + r"gadm36_IND_1_id.json", "w") as f:
    json.dump(states, f)

In [8]:
with open(data_in + r"gadm36_IND_0.json", "r") as f:
    districts = json.load(f)

In [9]:
for i in districts["features"]:
    i["id"] = i["properties"]["NAME_0"].lower()

In [10]:
with open(data_out + r"gadm36_IND_0_id.json", "w") as f:
    json.dump(districts, f)

## Rough

In [12]:
with open(data_out + r"gadm36_IND_3_id.json", "r") as f:
    blocks = json.load(f)

In [16]:
for i in blocks["features"]:
    if i["id"] == "islampur":
        print(i["id"])

islampur
islampur
